In [3]:
!pip install tensorflow==2.2.0 h5py==2.10.0 numpy==1.21 Pillow
!pip install -U coremltools

  Using cached tensorflow-2.2.0-cp37-cp37m-manylinux2010_x86_64.whl (516.2 MB)


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [5]:
# Set parameters
img_size = 224
batch_size = 32
classes = ['dhanesh','goutham','unknown']
num_classes = len(classes)  # Set this according to your number of classes

# Prepare the data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
validation_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    'dataset/test',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 37 images belonging to 3 classes.
Found 10 images belonging to 3 classes.


In [6]:
train_generator
validation_generator

In [7]:
# Load the MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



2024-07-14 21:37:47.877187: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-07-14 21:37:47.914134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-14 21:37:47.914376: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 5 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 104.43GiB/s
2024-07-14 21:37:47.914621: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-07-14 21:37:47.914695: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic librar

9412608/9406464 [==============================] - 3s 0us/step


In [8]:
# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
1/2 [==============>...............] - ETA: 0s - loss: 1.2253 - accuracy: 0.2000

2024-07-14 21:38:01.829046: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 154140672 exceeds 10% of free system memory.
2024-07-14 21:38:01.882843: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 156905472 exceeds 10% of free system memory.
2024-07-14 21:38:02.025678: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 57802752 exceeds 10% of free system memory.
2024-07-14 21:38:02.048773: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 57802752 exceeds 10% of free system memory.
2024-07-14 21:38:02.141631: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 57802752 exceeds 10% of free system memory.


2/2 [==============================] - 3s 1s/step - loss: 2.7863 - accuracy: 0.6216 - val_loss: 1.3861 - val_accuracy: 0.7000
Epoch 2/10
2/2 [==============================] - 2s 1s/step - loss: 0.6198 - accuracy: 0.7027 - val_loss: 0.0491 - val_accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 1s 271ms/step - loss: 0.0572 - accuracy: 1.0000 - val_loss: 0.7996 - val_accuracy: 0.7000
Epoch 4/10
2/2 [==============================] - 2s 994ms/step - loss: 0.6000 - accuracy: 0.8108 - val_loss: 0.8265 - val_accuracy: 0.7000
Epoch 5/10
2/2 [==============================] - 2s 951ms/step - loss: 0.1241 - accuracy: 0.9730 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 1s 272ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 2s 963ms/step - loss: 0.0320 - accuracy: 1.0000 - val_loss: 0.0148 - val_accuracy: 1.0000
Epoch 8/10
2/2 [=====================

In [51]:
# Save the model
model.save('model1.h5')

In [47]:
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
# Load and preprocess an input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale image
    return img_array

# Function to perform inference
def predict_image(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    print(predictions)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0], predictions[0]

# Get the class labels from the training data (assuming train_generator is still available)
# If not, you can save the class indices during training and load them here
class_labels = {v: k for k, v in train_generator.class_indices.items()}
print(train_generator.class_indices.items())
# Perform inference on a test image
img_path = 'dataset/test/dhanesh/snapshot_18.jpg'
predicted_class, prediction_probabilities = predict_image(img_path)

print(f"Predicted Class: {class_labels[predicted_class]}")
print(f"Prediction Probabilities: {prediction_probabilities}")

# Optional: Print class labels and corresponding probabilities
for idx, label in class_labels.items():
    print(f"{label}: {prediction_probabilities[idx]*100:.2f}%")

dict_items([('dhanesh', 0), ('goutham', 1), ('unknown', 2)])
[[1.0000000e+00 2.4493604e-10 8.0224133e-11]]
Predicted Class: dhanesh
Prediction Probabilities: [1.0000000e+00 2.4493604e-10 8.0224133e-11]
dhanesh: 100.00%
goutham: 0.00%
unknown: 0.00%


Infernce using opencv


In [10]:
import coremltools as ct
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Failed to load _MLModelProxy: No module named 'coremltools.libcoremlpython'


In [13]:
# Set parameters
img_size = 224
batch_size = 32
class_labels = [0,1,2]

loaded_model = './model1.h5'
final_model = './people4.mlmodel'

In [52]:
class_labels = ['dhanesh','goutham','unknown']
# Load the trained Keras model
keras_model = load_model(loaded_model)
# keras_model.save('face_recognition_mobilenet_savedmodel', save_format='.keras')
classifier_config = ct.ClassifierConfig(class_labels)
# Convert the model to Core ML
mlmodel = ct.convert(keras_model, inputs=[ct.ImageType(shape=(1, img_size, img_size, 3))],convert_to="neuralnetwork")

# Save the Core ML model
mlmodel.save(final_model)

2024-07-14 22:38:42.216700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-14 22:38:42.216941: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-14 22:38:42.217008: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2024-07-14 22:38:42.217374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-14 22:38:42.217505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 5 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 104.43GiB/s
2024-07-14 22:38:42.21

In [46]:
mlmodel.save("MobileNetV2.mlpackage")

In [44]:
# Use PIL to load and resize the image to expected size
from PIL import Image
example_image = Image.open("dataset/test/dhanesh/snapshot_18.jpg").resize((224, 224))

# Make a prediction using Core ML
out_dict = mlmodel.predict({"dhanesh": example_image})

# Print out top-1 prediction
print(out_dict["classLabel"])

Exception: Model prediction is only supported on macOS version 10.13 or later.

In [34]:
# Download class labels (from a separate file)
import urllib
label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
class_labels = urllib.request.urlopen(label_url).read().splitlines()
class_labels = class_labels[1:] # remove the first class which is background
assert len(class_labels) == 1000

# make sure entries of class_labels are strings
for i, label in enumerate(class_labels):
  if isinstance(label, bytes):
    class_labels[i] = label.decode("utf8")
print(type(class_labels))
print(class_labels)

<class 'list'>
['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead', 'electric ray', 'stingray', 'cock', 'hen', 'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco', 'indigo bunting', 'robin', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel', 'kite', 'bald eagle', 'vulture', 'great grey owl', 'European fire salamander', 'common newt', 'eft', 'spotted salamander', 'axolotl', 'bullfrog', 'tree frog', 'tailed frog', 'loggerhead', 'leatherback turtle', 'mud turtle', 'terrapin', 'box turtle', 'banded gecko', 'common iguana', 'American chameleon', 'whiptail', 'agama', 'frilled lizard', 'alligator lizard', 'Gila monster', 'green lizard', 'African chameleon', 'Komodo dragon', 'African crocodile', 'American alligator', 'triceratops', 'thunder snake', 'ringneck snake', 'hognose snake', 'green snake', 'king snake', 'garter snake', 'water snake', 'vine snake', 'night snake', 'boa constrictor', 'rock python', 'Indian cobra', 'green mamba', 'sea snake', 'horned viper', 

In [35]:
class_labels = ['dhanesh','goutham','unknown']
print(type(class_labels))
print(class_labels)

<class 'list'>
['dhanesh', 'goutham', 'unknown']


In [ ]:
import coremltools as ct

# Replace 'YourModel.mlmodel' with the path to your model file
model = ct.models.MLModel(final_model)

In [ ]:
from PIL import Image
import numpy as np
img_path = 'dataset/test/dhanesh/snapshot_18.jpg'
# Load and preprocess your image
image = Image.open(img_path)
image = image.resize((224, 224)) # Resize to match model's input size
image = np.array(image) / 255.0  # Normalize pixel values

# If your model expects a specific shape, reshape the array
image = image.reshape(1, 224, 224, 3)  # Example for a batch of 1 image

In [ ]:
# Assuming your model has a single input named 'image'
predictions = model.predict({'image': image})

print(predictions)